### Single Document Agent

In this exercise, we will down load the latest AAPL call earnings transcript and then we will setup an agent to ask relevant questions - like 
<li> what are potential macro challenges </li>
<li> which product segment saw a big jump in revenue </li>
<li> which product segment didn't perform as expected </li>
<li>Identify any mentions of cost-cutting or operational efficiency measures.</li>
<li>Conduct a sentiment analysis of the CEO's remarks during the most recent earnings call.</li>
<li>Identify any discussions about entering new markets or new products </li>
<li>any changes in the executive team announced </li>
<li>any changes in corporate strategy </li>
<li>any mentions of debt restructuring or financing activitie</li>
<li>any mentions of special dividends or one-time payouts</li>

In [2]:
%pip install -Uq llama-index

Note: you may need to restart the kernel to use updated packages.


In [21]:
from pathlib import Path
import requests
import os
import getpass


from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)

from llama_parse import LlamaParse


#### Download APPL earning calls transcript (Q3 2024)

I am using RapidAPI.  You would need to sign up for a free account and use RapidAPI  API KEY

In [15]:
url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/get-details"

# https://seekingalpha.com/article/4709458-apple-inc-aapl-q3-2024-earnings-call-transcript
querystring = {"id":"4709458"}

headers = {
	"x-rapidapi-key": "653ba3e057msh1e063e0e688815cp155928jsn49c8b4e7c2e9",
	"x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring).json()
call_content = earlings_call["data"]["attributes"]["content"]
ticker= earlings_call["included"][1]["attributes"]["name"]

data_path = Path("data")
if not data_path.exists():
    Path.mkdir(data_path)

with open(data_path / f"{ticker}.txt", "w") as fp:
    fp.write(call_content)

In [23]:
# LlamaParse is async-first, so we need to run this line of code if you are working on a notebook
import nest_asyncio

nest_asyncio.apply()

In [19]:
os.environ['OPENAI_API_KEY'] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key:  ········


In [20]:
os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("Llama Cloud API Key: ")

 ········


In [25]:
# Load document
documents = LlamaParse(result_type="text").load_data("./data/AAPL.txt")
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


Started parsing the file under job_id b55109c0-512d-4f09-8dfa-e1e4ae1475f7


In [26]:
# ask question
response = query_engine.query("any changes in corporate strategy")
print(response)


There have been discussions about potential changes in the corporate strategy, particularly in terms of addressing foreign exchange headwinds and focusing on growing Services double digits. Additionally, there is a mention of integrating with ChatGPT for world knowledge, indicating a potential shift towards leveraging AI technologies for knowledge-related initiatives.


In [27]:
# ask question
response = query_engine.query("what are potential macro challenges mentioned")
print(response)

The potential macro challenges mentioned include the competitive environment in China being the most competitive in the world, macroeconomic factors affecting the market, and the impact of the state of commodity markets on the business.


In [28]:
# ask question
response = query_engine.query("which product segment saw a big jump in revenue and which product segment didn't perform as expected")
print(response)

The Services segment saw a big jump in revenue, while the Mac segment did not perform as expected.
